In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#scripts
from src import rank_metrics
from src import helpers
from src import table_encoder
from src import metrics
    
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql

from pyspark.ml.recommendation import ALS
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer



In [2]:
data_path = '../data/csv'

In [3]:
ratings_df, movies_df, encoded_movies_df, tags_df, enoded_tags_df = helpers.load_format_data(data_path)

In [4]:
train, test = helpers.load_format_data_for_model(data_path)

In [5]:
# model = helpers.fit_als(train)

In [6]:
# helpers.save_model('als.model')

In [7]:
model = helpers.load_model('als.model')

In [8]:
allUserRecs, allMovieRecs = helpers.predict_for_all_users(model, test)

RMSE = 0.6517201753575874


In [9]:
user_prediction = helpers.predict_for_one_user(model, 5, test, movies_df)
user_prediction

['Shawshank Redemption, The (1994)',
 'Interview with the Vampire: The Vampire Chronicles (1994)',
 'Shadowlands (1993)',
 'Batman Forever (1995)']

In [10]:
user_liked_movie_ids = helpers.get_user_liked_movie_ids(5, ratings_df)

In [11]:
user_liked_movie_titles = helpers.get_user_liked_movie_titles(5, ratings_df, movies_df)[:5]

In [12]:
rec_movie_ids = helpers.get_ids_from_titles(user_prediction, movies_df)
rec_movie_ids

[318, 253, 534, 153]

In [13]:
metrics.compare_recs_to_user_likes(rec_movie_ids, user_liked_movie_ids, encoded_genres=encoded_movies_df)

,movie ids,similarity
0,318,0.213679
1,253,0.158436
2,534,0.185038
3,153,0.375682


In [14]:
metrics.compare_movie(232, 984, encoded_movies_df)

0.8165

In [31]:
sample = helpers.format_ratings(ratings_df[ratings_df['userId']==1])

/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
sample.select("*").toPandas()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
227,1,3744,4.0
228,1,3793,5.0
229,1,3809,4.0
230,1,4006,4.0


In [17]:
sample

DataFrame[userId: bigint, movieId: bigint, rating: double]

In [38]:
sample = [[611, 1, 5],
[611, 356, 5],
[611, 318, 5],
[611, 593, 5],
[611, 296, 5],
[611, 2571, 5],
[611, 2858, 5],
[611, 344, 5],
[611, 480, 5],
[611, 110, 5]]

In [39]:
sample = pd.DataFrame(sample, columns=['userId', 'movieId', 'rating'])
sample

,userId,movieId,rating
0,611,1,5
1,611,356,5
2,611,318,5
3,611,593,5
4,611,296,5
5,611,2571,5
6,611,2858,5
7,611,344,5
8,611,480,5
9,611,110,5


In [40]:
sample = helpers.format_ratings(sample)

In [41]:
model.transform(sample).select("*").toPandas()

,userId,movieId,rating,prediction


In [32]:
predictions = model.transform(sample)
prediction_ids = helpers.spark_to_pandas(predictions).sort_values('prediction',
                                                              ascending=False)[:5]['movieId']
prediction_titles = []
for id in prediction_ids:
    title = movies_df[movies_df['movieId'] == id]['title']
    prediction_titles.append(title.values[0])

predictions.select("*").toPandas().sort_values('prediction',ascending=False)[:5]['movieId']

214    3448
70     1198
69     1197
68     1196
114    1927
Name: movieId, dtype: int64

In [33]:
prediction_titles

['Good Morning, Vietnam (1987)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Princess Bride, The (1987)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'All Quiet on the Western Front (1930)']

In [19]:
us = pd.Series([1])
movieIds = pd.Series([1])
ratings = pd.Series([1])

df = pd.DataFrame()
df['userId'] = us
df['movieId'] = movieIds
df['rating'] = ratings

[autoreload of src.helpers failed: Traceback (most recent call last):
  File "/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/sam/flatiron/projects/mod4/MovieLens-Rec-Sys/src/helpers.py", line 19, in <module>
    sc = Spar

In [24]:
df

,userId,movieId,rating
0,1,1,1


In [ ]:
diction = {'blast':5,
          'cop':0.5,
          'toys':0}

In [ ]:
tits = list(diction.keys())
rats = list(diction.values())
us = [9999]*len(tits)
movieIds = []
for title in tits:
    movieIds.append(movies_df[movies_df['title']==title]['movieId'])

In [ ]:
movieIds = pd.Series(movieIds)
rats = pd.Series(rats)
us = pd.Series(us)

In [ ]:
df = pd.DataFrame()

In [ ]:
movies_df[movies_df['title']=='Toy Story (1995)']['movieId']

In [ ]:
df['userId'] = us
df['movieId'] = movieIds
df['rating'] = rats


In [ ]:
df

In [ ]:
us, rats, tits